In [60]:
from Dataloader import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pkuseg
import gensim
from tkinter import _flatten
from numpy import mat

In [61]:
# 加载数据
loader = Dataloader('../启明学院智能客服知识库.xlsx', '知识库')

In [64]:
#处理加载的数据，建立索引，去重
sentince_list = []
sentince_index = []
for st in loader._data_pool:
    sentince_list.append(st[0])
    sentince_list.append(st[1])
    sentince_index.append((st[0],st[2]))
    sentince_index.append((st[1],st[2]))
sentince_list = list(set(sentince_list))
sentince_index = list(set(sentince_index))
sentince_index = {each[0]:each[1] for each in sentince_index}

In [65]:
#使用TF-IDF将句子转化为向量形式,形成矩阵
seg = pkuseg.pkuseg()           # 以默认配置加载模型
tfidf_vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", max_df=0.7)

real_documents = []
for item_text in sentince_list:
    item_str = seg.cut(item_text)
    real_documents.append(item_str)
document = [" ".join(sent0) for sent0 in real_documents]

tfidf_model = tfidf_vectorizer.fit(document)
#print(tfidf_model.vocabulary_)

#print(tfidf_vectorizer.idf_) #特征对应的权重
#print(tfidf_vectorizer.get_feature_names())#特征词
#print(real_vec.toarray()) #上面对话对应的向量表示
sparse_result = tfidf_model.transform(document).todense()

In [72]:
#测试句子转换
test_sentices = ['启明学院的领导人是谁？']
test_documents = []
for item_text in test_sentices:
    item_str = seg.cut(item_text)
    test_documents.append(item_str)
test_document = [" ".join(sent0) for sent0 in test_documents]
result = tfidf_model.transform(test_document).todense()


In [73]:
#结果测试
ans = result*sparse_result.T
ans = ans.argmax(axis=1)
answer = sentince_list[ans.tolist()[0][0]]
answer = loader.get_answer(sentince_index[answer])

In [74]:
print(answer)

学院现任领导院长：梁亮胜、陈刚、黄刚
